# EDA – Emisii, mix energetic și variabile sociodemografice (NUTS2, CEE)

Acest notebook face o analiză exploratorie (EDA) pe setul `energy_socio_merged.csv` (date NUTS2 + energie + socio), fără a exporta tabele CSV intermediare.
Mă utilizez de datele colectate și curățate de către Osaci Cosmin, în plus datele curățate de Ariana Vlaicu (sociodemografice) și analizate. Cu colaborarea lor și a lui Mihai Iacob (feature engineering și colectare de date), acest capitol de notebook a fost scris de Ioana Zaharie.

## Obiective
- verificare structură panel (ani, regiuni, acoperire).
- missingness și calitatea datelor.
- statistici descriptive și distribuții (skew/outliers).
- diferențe între grupuri (lock-in binar) + teste (ANOVA, Kruskal–Wallis) și asumpții.
- corelații (matrici Pearson/Spearman) + teste Pearson (r, p-value).
- teste de normalitate (Kolmogorov–Smirnov; orientativ).
- modele: OLS (pooled vs two-way FE cu SE cluster pe NUTS2) și regresie logistică (variabila co2_high).

> Notă: Pentru panel, testele clasice pe observații anuale presupun independență; de aceea modelele cu efecte fixe și erori clusterizate sunt mai potrivite pentru inferență.


In [53]:
# Setup
import pandas as pd
import numpy as np

from scipy import stats
import statsmodels.api as sm
import statsmodels.formula.api as smf

pd.set_option('display.max_columns', 200)
pd.set_option('display.width', 180)

energy = pd.read_csv("CEE_NUTS2_energy_emissions_panel_ECE.csv")
socio  = pd.read_csv("nuts2_basic_panel_clean.csv")

socio = socio.rename(columns={"geo": "nuts2", "time": "year"})

merged = energy.merge(socio, on=["nuts2", "year"], how="left")
merged.to_csv("energy_socio_merged.csv", index=False) 

DATA_PATH = "energy_socio_merged.csv"
df = pd.read_csv(DATA_PATH)

def p_to_stars(p):
    if pd.isna(p):
        return ""
    if p < 0.001:
        return "***"
    elif p < 0.01:
        return "**"
    elif p < 0.1:
        return "*"
    else:
        return ""

def fmt_coef_with_sig(coef, p, digits=3):
    if pd.isna(coef):
        return ""
    return f"{coef:.{digits}f}{p_to_stars(p)}"


## 1) Încărcare date

Încărcăm datasetul deja merge-uit. Cheia este `nuts2` + `year`.


In [55]:
df = pd.read_csv(DATA_PATH)

print('Rows:', len(df), 'Cols:', df.shape[1])
print('Ani:', int(df['year'].min()), '-', int(df['year'].max()))
print('NUTS2:', df['nuts2'].nunique(), 'ISO:', df['ISO'].nunique())

df.head()

Rows: 2244 Cols: 31
Ani: 1990 - 2022
NUTS2: 68 ISO: 12


,Substance,ISO,Country,nuts2,NUTS 2 desc,Sector,year,co2_energy,country,gas,oil,renewables,solid_fossil,solid_fuels,fossil_share,renewable_share,coal_share,co2_growth,delta_renewables,delta_coal,coal_lockin,gdp,gfcf,gerd,employment_total,gva_total,population_total,hightech_emp,gfcf_over_gdp,gerd_over_gdp,hightech_emp_share
0,CO2,AUT,Austria,AT11,Burgenland,Energy,1990,240.746162,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,CO2,AUT,Austria,AT11,Burgenland,Energy,1991,264.931007,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.100458,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,CO2,AUT,Austria,AT11,Burgenland,Energy,1992,272.857652,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.029920,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,CO2,AUT,Austria,AT11,Burgenland,Energy,1993,289.490577,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.060958,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,CO2,AUT,Austria,AT11,Burgenland,Energy,1994,314.739180,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.087217,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## 2) Structura panel (observații pe ani și pe regiuni)

Ne interesează dacă panelul e relativ echilibrat (aceeași regiune apare în majoritatea anilor) și dacă există ani cu acoperire slabă.


In [56]:
# Observații pe ani
year_counts = df['year'].value_counts().sort_index()
print('Obs/an (primele 10):')
print(year_counts.head(10))
print('Obs/an (ultimele 10):')
print(year_counts.tail(10))

# Observații pe regiune
reg_counts = df['nuts2'].value_counts()
print('Obs/regiune (descriere):')
print(reg_counts.describe())


Obs/an (primele 10):
year
1990    68
1991    68
1992    68
1993    68
1994    68
1995    68
1996    68
1997    68
1998    68
1999    68
Name: count, dtype: int64
Obs/an (ultimele 10):
year
2013    68
2014    68
2015    68
2016    68
2017    68
2018    68
2019    68
2020    68
2021    68
2022    68
Name: count, dtype: int64
Obs/regiune (descriere):
count    68.0
mean     33.0
std       0.0
min      33.0
25%      33.0
50%      33.0
75%      33.0
max      33.0
Name: count, dtype: float64


## 3) Missingness (lipsuri)

- Identificăm coloanele cu multe lipsuri.
- Pentru variabilele socio, ne așteptăm ca acoperirea să fie în special după 2008.


In [57]:
miss_pct = df.isna().mean().sort_values(ascending=False) * 100
print('Top 20 coloane cu lipsuri (%):')
print(miss_pct.head(20))

# Acoperire socio pe ani (proxy: GDP disponibil)
df['has_socio'] = ~df['gdp'].isna()
cover = df.groupby('year')['has_socio'].mean().round(3)
print('Acoperire sociodemografica pe ani (share_with_socio):')
print(cover)


Top 20 coloane cu lipsuri (%):
employment_total      54.545455
gva_total             54.545455
population_total      54.545455
hightech_emp          54.545455
gfcf_over_gdp         54.545455
hightech_emp_share    54.545455
gerd_over_gdp         54.545455
gerd                  54.545455
gdp                   54.545455
gfcf                  54.545455
delta_coal            20.142602
delta_renewables      20.142602
fossil_share          17.647059
solid_fossil          17.647059
solid_fuels           17.647059
country               17.647059
gas                   17.647059
oil                   17.647059
renewables            17.647059
coal_share            17.647059
dtype: float64
Acoperire sociodemografica pe ani (share_with_socio):
year
1990    0.0
1991    0.0
1992    0.0
1993    0.0
1994    0.0
1995    0.0
1996    0.0
1997    0.0
1998    0.0
1999    0.0
2000    0.0
2001    0.0
2002    0.0
2003    0.0
2004    0.0
2005    0.0
2006    0.0
2007    0.0
2008    1.0
2009    1.0
2010    1.0
201

## 4) Statistici descriptive

Calculăm descriptive pentru variabilele cheie. Skew mare sugerează distribuții asimetrice și outliers (frecvent pentru emisii și variabile macro).


In [58]:
vars_focus = [
    'co2_energy','co2_growth','coal_lockin',
    'coal_share','renewable_share','fossil_share',
    'gdp','gva_total','population_total','employment_total',
    'hightech_emp','hightech_emp_share','gerd_over_gdp'
]
vars_focus = [v for v in vars_focus if v in df.columns]

desc = df[vars_focus].describe().T
desc['skew'] = df[vars_focus].skew(numeric_only=True)
print(desc[['count','mean','std','min','25%','50%','75%','max','skew']])


                     count          mean           std           min            25%            50%           75%           max       skew
co2_energy          2244.0  5.446252e+03  7.679266e+03     12.628300     877.407644    2508.946187  6.404323e+03  4.634213e+04   2.588499
co2_growth          2176.0  8.999757e-03  5.695943e-01     -0.958559      -0.076110      -0.020140  3.658794e-02  1.672989e+01  22.281607
coal_lockin         1848.0  2.131843e+00  1.777767e+00      0.007992       0.722598       1.476546  3.489256e+00  6.130194e+00   0.732770
coal_share          1848.0  1.150220e-01  7.311529e-02      0.000390       0.049919       0.111486  1.788661e-01  2.529111e-01   0.143386
renewable_share     1848.0  1.247963e-01  8.139971e-02      0.010445       0.069857       0.107555  1.606239e-01  6.838959e-01   1.736357
fossil_share        1848.0  7.468562e-01  8.749709e-02      0.238064       0.712580       0.752676  7.923647e-01  9.201572e-01  -1.254561
gdp                 1020.0  7.7526

## 5) Diferențe între grupuri: lock-in (binar)

Construim un grup binar `coal_lockin_bin` (0/1) pe baza medianei lui `coal_lockin`.

Apoi:
- descriptive pe grup pentru `co2_energy`
- ANOVA (benchmark)
- teste asumpții (Shapiro pe reziduuri, Levene)
- Kruskal–Wallis (pre-regresie)


In [59]:
DV = 'co2_energy'
work = df.copy()

med_lock = work['coal_lockin'].median(skipna=True)
work['coal_lockin_bin'] = np.where(work['coal_lockin'].isna(), np.nan, (work['coal_lockin'] >= med_lock).astype(int))
sub = work[[DV,'coal_lockin_bin']].dropna()

print('N observații în test:', len(sub))
print('Descriptive CO2 pe grup (0/1):')
print(sub.groupby('coal_lockin_bin')[DV].agg(['count','mean','median','std','min','max']))

# ANOVA
anova_model = smf.ols(f"{DV} ~ C(coal_lockin_bin)", data=sub).fit()
print('ANOVA:')
print(sm.stats.anova_lm(anova_model, typ=2))

# Asumpții: Shapiro pe reziduuri (sample max 5000)
resid = anova_model.resid
resid_samp = resid.sample(min(5000, len(resid)), random_state=0)
sh_W, sh_p = stats.shapiro(resid_samp)

# Levene (median-centered)
lev_stat, lev_p = stats.levene(
    sub[sub['coal_lockin_bin']==0][DV],
    sub[sub['coal_lockin_bin']==1][DV],
    center='median'
)
print('Teste asumpții (ANOVA):')
print({'shapiro_W': sh_W, 'shapiro_p': sh_p, 'levene_stat': lev_stat, 'levene_p': lev_p})

# Kruskal–Wallis
H, p = stats.kruskal(sub[sub['coal_lockin_bin']==0][DV], sub[sub['coal_lockin_bin']==1][DV])
print('Kruskal–Wallis:', {'H': H, 'p_value': p})


N observații în test: 1848
Descriptive CO2 pe grup (0/1):
                 count         mean       median          std        min          max
coal_lockin_bin                                                                      
0.0                924  4897.395956  2614.254513  6875.835826  12.628300  46342.13004
1.0                924  7731.865415  3987.579903  9078.982231  81.439611  44367.55680
ANOVA:
                          sum_sq      df          F        PR(>F)
C(coal_lockin_bin)  3.711808e+09     1.0  57.234606  6.065225e-14
Residual            1.197177e+11  1846.0        NaN           NaN
Teste asumpții (ANOVA):
{'shapiro_W': np.float64(0.733881893406418), 'shapiro_p': np.float64(2.1380684952949776e-47), 'levene_stat': np.float64(41.94739993189692), 'levene_p': np.float64(1.1989635840100708e-10)}
Kruskal–Wallis: {'H': np.float64(75.51128845606237), 'p_value': np.float64(3.633237722375314e-18)}


## 6) Corelații (2008+)

Folosim `2008+` pentru că atunci sunt disponibile variabilele socio.

- Matrice Pearson și Spearman pe un subset de variabile.
- Teste Pearson (r, p-value) între `co2_energy` și câțiva predictori.


In [60]:
df08 = df[df['year'] >= 2008].copy()

vars_corr = [
    'co2_energy','coal_lockin','coal_share','renewable_share','fossil_share',
    'gdp','population_total','hightech_emp_share','gerd_over_gdp'
]
vars_corr = [v for v in vars_corr if v in df08.columns]

num08 = df08[vars_corr].dropna()

print('N pentru corelații (complete cases pe vars_corr):', len(num08))

pear = num08.corr(method='pearson')
spear = num08.corr(method='spearman')

print('Pearson:')
print(pear.round(3))
print('Spearman:')
print(spear.round(3))

# Teste Pearson (co2 vs predictori)
rows=[]
for v in [x for x in vars_corr if x != 'co2_energy']:
    tmp = df08[['co2_energy', v]].dropna()
    if len(tmp) >= 3:
        r, pv = stats.pearsonr(tmp['co2_energy'], tmp[v])
        rows.append({'x': v, 'n': len(tmp), 'pearson_r': r, 'p_value': pv})

pear_tests = pd.DataFrame(rows).sort_values('p_value')
print('Teste Pearson (co2_energy vs predictori):')
print(pear_tests)


N pentru corelații (complete cases pe vars_corr): 840
Pearson:
                    co2_energy  coal_lockin  coal_share  renewable_share  fossil_share    gdp  population_total  hightech_emp_share  gerd_over_gdp
co2_energy               1.000        0.349       0.329           -0.064        -0.055 -0.135             0.359              -0.137         -0.066
coal_lockin              0.349        1.000       0.811           -0.003        -0.176 -0.278             0.144              -0.220          0.122
coal_share               0.329        0.811       1.000           -0.261        -0.004 -0.324             0.103              -0.224         -0.062
renewable_share         -0.064       -0.003      -0.261            1.000        -0.918 -0.133            -0.081              -0.029          0.205
fossil_share            -0.055       -0.176      -0.004           -0.918         1.000  0.257             0.039               0.100         -0.219
gdp                     -0.135       -0.278      -0.324

## 7) Normalitate (Kolmogorov–Smirnov) – orientativ

Testăm dacă variabilele (standardizate) seamănă cu o distribuție normală.

> Notă: KS este sensibil la eșantioane mari; se folosește aici ca semnal, nu ca „verdict final”.


In [61]:
df08 = df[df['year'] >= 2008].copy()

ks_vars = ['co2_energy','co2_growth','coal_lockin','coal_share','renewable_share','gdp']
ks_vars = [v for v in ks_vars if v in df08.columns]

rows=[]
for v in ks_vars:
    x = df08[v].dropna()
    if len(x) >= 8:
        z = (x - x.mean())/(x.std(ddof=1) + 1e-12)
        D, pv = stats.kstest(z, 'norm')
        rows.append({'variable': v, 'n': len(x), 'ks_D': D, 'p_value': pv})

ks_tbl = pd.DataFrame(rows).sort_values('p_value')
print(ks_tbl)


          variable     n      ks_D        p_value
5              gdp  1020  0.357218  5.270467e-117
1       co2_growth  1020  0.282679   1.331756e-72
0       co2_energy  1020  0.251453   2.550540e-57
4  renewable_share   840  0.233546   8.920658e-41
2      coal_lockin   840  0.160215   2.646620e-19
3       coal_share   840  0.116301   2.351123e-10


## 8) OLS pe panel (2008+)

Estimăm două modele pentru `log(co2_energy + 1)`:

1) **Pooled OLS** (benchmark) cu erori robuste HC3.
2) **Two-way fixed effects (TWFE)**: `C(nuts2) + C(year)` + erori **cluster** pe `nuts2`.

> Interpretare: dacă semnele/p-urile diferă mult între pooled și TWFE, e un indiciu că efectele sunt mai degrabă cross-sectional (diferențe între regiuni) decât within (schimbări în aceeași regiune). Referință la Bourdin & Perrot, 2025.


In [ ]:
df08 = df[df['year'] >= 2008].copy()
df08['log_co2'] = np.log(df08['co2_energy'] + 1)

covs = ['coal_lockin','renewable_share','coal_share','gdp','population_total','hightech_emp_share','gerd_over_gdp']
covs = [c for c in covs if c in df08.columns]

model_df = df08[['log_co2','nuts2','year'] + covs].dropna().copy()
print('N obs model:', len(model_df), '| regiuni:', model_df['nuts2'].nunique(), '| ani:', model_df['year'].nunique())

formula = 'log_co2 ~ ' + ' + '.join(covs)

pooled = smf.ols(formula, data=model_df).fit(cov_type='HC3')
twfe = smf.ols(formula + ' + C(nuts2) + C(year)', data=model_df).fit(
    cov_type='cluster', cov_kwds={'groups': model_df['nuts2']}
)

print('Pooled OLS (HC3) – coeficienți:')
print(pooled.summary().tables[1])

# extragem doar termenii principali (fără dummies)
params = twfe.params
bse = twfe.bse
pvals = twfe.pvalues
main = ~params.index.str.startswith('C(')

print('TWFE (cluster pe nuts2) – termeni principali:')
print(pd.DataFrame({'term': params.index[main], 'coef': params.values[main], 'se': bse.values[main], 'p_value': pvals.values[main]}))

print('Fit:')
print(pd.DataFrame([
    {'model':'pooled_HC3', 'nobs': int(pooled.nobs), 'r2': pooled.rsquared, 'adj_r2': pooled.rsquared_adj},
    {'model':'twfe_cluster_nuts2', 'nobs': int(twfe.nobs), 'r2': twfe.rsquared, 'adj_r2': twfe.rsquared_adj}
]))



N obs model: 840 | regiuni: 56 | ani: 15
Pooled OLS (HC3) – coeficienți:
                         coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------------
Intercept              6.3323      0.585     10.822      0.000       5.185       7.479
coal_lockin            0.1404      0.039      3.629      0.000       0.065       0.216
renewable_share        1.2771      0.567      2.254      0.024       0.167       2.388
coal_share             5.6485      1.344      4.201      0.000       3.013       8.284
gdp                -1.362e-08   1.79e-08     -0.762      0.446   -4.86e-08    2.14e-08
population_total       0.0002   2.65e-05      8.376      0.000       0.000       0.000
hightech_emp_share    -0.5515      0.498     -1.108      0.268      -1.527       0.424
gerd_over_gdp         -2.5702      2.984     -0.861      0.389      -8.420       3.279
TWFE (cluster pe nuts2) – termeni principali:
           

c:\Users\ioana\OneDrive\Desktop\ADC 2025\12. DSW\.venv\Lib\site-packages\statsmodels\base\model.py:1894: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 7, but rank is 6
  warnings.warn('covariance of constraints does not have full '


## 9) Regresie logistică (2008+): probabilitatea de emisii mari

Definim `co2_high` astfel:
- dacă există deja în dataset, îl folosim;
- altfel îl construim ca `co2_energy >= median(co2_energy)` (pe 2008+).

Apoi estimăm:
1) Logit pooled (HC3)
2) Logit cu **FE pe ani**: `+ C(year)` (HC3)

> Notă: FE pe nuts2 în logit e mai dificil (incidental parameter / separare); de aceea aici includem doar FE pe ani ca variantă simplă.


In [63]:
df08 = df[df['year'] >= 2008].copy()

# outcome binar
if 'co2_high' in df08.columns:
    df08['y_bin'] = df08['co2_high']
else:
    df08['y_bin'] = (df08['co2_energy'] >= df08['co2_energy'].median()).astype(int)

preds = ['coal_lockin','renewable_share','coal_share','gdp','population_total','hightech_emp_share','gerd_over_gdp']
preds = [p for p in preds if p in df08.columns]

# log-transform pe variabile de scară (opțional)
if 'gdp' in preds:
    df08['log_gdp'] = np.log(df08['gdp'] + 1)
    preds = [p if p != 'gdp' else 'log_gdp' for p in preds]
if 'population_total' in preds:
    df08['log_pop'] = np.log(df08['population_total'] + 1)
    preds = [p if p != 'population_total' else 'log_pop' for p in preds]

logit_df = df08[['y_bin','year'] + preds].dropna().copy()
print('N obs logit:', len(logit_df))

logit_formula = 'y_bin ~ ' + ' + '.join(preds)

logit_pooled = smf.logit(logit_formula, data=logit_df).fit(disp=0, cov_type='HC3')
logit_year = smf.logit(logit_formula + ' + C(year)', data=logit_df).fit(disp=0, cov_type='HC3')

# tabel cu odds ratios

def logit_table(res):
    params = res.params
    se = res.bse
    pv = res.pvalues
    OR = np.exp(params)
    lo = np.exp(params - 1.96 * se)
    hi = np.exp(params + 1.96 * se)
    return pd.DataFrame({'term': params.index, 'coef': params.values, 'se': se.values, 'p_value': pv.values,
                         'odds_ratio': OR.values, 'or_ci_low': lo.values, 'or_ci_high': hi.values})

print('Logit pooled – odds ratios:')
print(logit_table(logit_pooled))

print('Logit + FE pe ani – termeni principali:')
lt = logit_table(logit_year)
print(lt[~lt['term'].str.startswith('C(')])

print('Fit (pseudo R2):')
print(pd.DataFrame([
    {'model':'logit_pooled_HC3', 'nobs': int(logit_pooled.nobs), 'llf': float(logit_pooled.llf), 'pseudo_r2': float(logit_pooled.prsquared)},
    {'model':'logit_yearFE_HC3', 'nobs': int(logit_year.nobs), 'llf': float(logit_year.llf), 'pseudo_r2': float(logit_year.prsquared)}
]))




N obs logit: 840
Logit pooled – odds ratios:
                 term       coef        se       p_value    odds_ratio     or_ci_low    or_ci_high
0           Intercept -19.969957  2.804027  1.064610e-12  2.124017e-09  8.716100e-12  5.175996e-07
1         coal_lockin   0.145579  0.077992  6.195799e-02  1.156710e+00  9.927417e-01  1.347760e+00
2     renewable_share  -0.582798  1.190002  6.243140e-01  5.583338e-01  5.419304e-02  5.752336e+00
3          coal_share   7.180507  2.724630  8.403661e-03  1.313574e+03  6.298002e+00  2.739722e+05
4             log_gdp  -0.199103  0.083636  1.728513e-02  8.194654e-01  6.955655e-01  9.654353e-01
5             log_pop   2.166105  0.259898  7.785793e-17  8.724235e+00  5.242012e+00  1.451967e+01
6  hightech_emp_share   4.194345  0.899010  3.078462e-06  6.631025e+01  1.138486e+01  3.862189e+02
7       gerd_over_gdp  12.308904  6.294780  5.053417e-02  2.216608e+05  9.715475e-01  5.057242e+10
Logit + FE pe ani – termeni principali:
                  term  

## Rezultate

Panelul NUTS2 utilizat este echilibrat pentru variabilele energetice (acoperire completă pe regiuni și ani), însă variabilele socio-demografice sunt disponibile consistent doar pentru perioada 2008–2022, motiv pentru care analizele care includ determinanți socio-economici au fost realizate pe acest sub-interval. 

Distribuțiile variabilelor cheie (în special emisiile CO2 și PIB) sunt puternic asimetrice și se abat de la normalitate, ceea ce justifică utilizarea transformărilor (de tip log) și a metodelor robuste/non-parametrice în EDA. Analiza comparativă pe grupuri arată diferențe semnificative între regiunile cu lock-in ridicat versus scăzut, regiunile cu lock-in mai mare având în medie și mediană emisii CO2 mai ridicate; totuși, deoarece ipotezele ANOVA (normalitate și omogenitatea varianțelor) sunt încălcate, rezultatele au fost confirmate printr-un test non-parametric (Kruskal–Wallis). 

Corelațiile bivariate indică asocieri pozitive între emisiile CO2 și indicatorii legați de dependența de cărbune (coal lock-in/coal share), precum și o asociere negativă cu ponderea ocupării în sectoare high-tech, în timp ce legătura cu ponderea regenerabilelor este mai slabă la nivel bivariant. În modelele de regresie, estimările pooled OLS surprind în principal diferențe între regiuni, însă specificația two-way fixed effects (efecte fixe pe regiune și pe an, cu erori clusterizate pe regiune) modifică semnificația unor coeficienți, sugerând că o parte din relațiile observate inițial sunt explicate de heterogenitatea invariantă în timp și de șocurile comune pe ani. 

În ansamblu, EDA susține existența unui gradient emisii–lock-in la nivel descriptiv, dar arată că inferența robustă necesită modele panel (TWFE/erori clusterizate) și o selecție atentă a predictorilor pentru a limita colinearitatea (în special între variabilele de tip „share” și indicatorii de „mărime” regională).